# Bookyland app

- Context:
  Bookyland app is a fake system that is a web and mobile app where you can buy books, this is notebook is a simulation of book purchases by users that are registered in the system

# Resources

- Users data API 
  - [User Random API] (https://randomuser.me/api/)

- Dollar USD vs MXN API
  - (API Banxico) - (https://www.banxico.org.mx/SieAPIRest/)

- Books are obtained from google books API
  - (Google books API) (https://www.googleapis.com/books/v1/volumes?q=)


In [154]:
import requests
import pandas as pd
import random
import time
import datetime
import uuid
import numpy as np

In [155]:
ALL_USERS = []


def fetch_random_user():
    time.sleep(1)  # Simulate delay
    response = requests.get('https://randomuser.me/api/')
    data = response.json()['results'][0]
    user_id = data['login']['uuid']
    user_name = data['name']['first']
    user_lastname = data['name']['last']
    user_country = data['location']['country']
    user_age = data['dob']['age']
    user_city = data['location']['city']
    user_registration_date = set_random_date(year_start=2019, year_end=2025)
    return {
        'user_id': user_id,
        'user_name': user_name,
        'user_lastname': user_lastname,
        'user_country': user_country,
        'user_age': user_age,
        'user_city': user_city,
        'user_registration_date': user_registration_date
    }


def fetch_random_book():
    time.sleep(1)  # Simulate delay
    categories = ['Horror', 'comedy', 'Mystery', 'Films', 'Healthy', 'Sports', 'Business', 'History',
                  'Fiction', 'Science', 'Maths', 'Science Fiction', 'Novels', 'Erotic', 'Drama', 'Comic']
    category_query = random.choice(categories)
    response = requests.get(
        f'https://www.googleapis.com/books/v1/volumes?q={category_query}')
    data = response.json()
    book = random.choice(data['items'])
    book_id = book['id']
    book_title = book['volumeInfo']['title']
    book_price = random.uniform(5, 50)
    book_editorial = book['volumeInfo']['publisher']
    book_page_length = book['volumeInfo']['pageCount']
    book_genre = book['volumeInfo'].get('categories', ['Unknown'])[0]
    book_language = book['volumeInfo']['language']
    book_mode = random.choice(['PDF', 'EBOOK', 'EPUB', 'PHYSICAL','AUDIO_BOOK'])
    book_author = book['volumeInfo'].get('authors', ['Unknown'])[0]
    book_isbn = book['volumeInfo'].get(
        'industryIdentifiers', [{'identifier': 'Unknown'}])[0]['identifier']
    return book_id, book_title, book_price, book_editorial, book_genre, book_author, book_isbn, book_page_length, book_language, book_mode,


def set_random_date(year_start: int, year_end: int):
    start = datetime.datetime(year_start, 1, 1)
    end = datetime.datetime(year_end, 1, 1)

    return start + datetime.timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())))


def fetch_exchange_rate():
    today_date = datetime.datetime.now().strftime("%Y-%m-%d")
    start_date, end_date = today_date, today_date
    token = '6777614a19ccff136423fd9eb05bc2a36c0a2eb0058c67214c67c1f77073948e'
    idSerie = 'SF43718'
    api_url = f'https://www.banxico.org.mx/SieAPIRest/service/v1/series/{idSerie}/datos/{start_date}/{end_date}'
    response = requests.get(api_url, headers={
        'Bmx-Token': token
    })
    data = response.json()
    exchange = data['bmx']['series'][0]['datos'][0]['dato']
    return exchange


def set_exchange_usd_to_mxn(book_price):
    exchange_rate = fetch_exchange_rate()
    if exchange_rate != 'N/E':
        dollar_price = float(exchange_rate)
        return round(float(book_price) * dollar_price, 2)
    else:
        return exchange_rate


def simulate_book_purchases(num_purchases):
    if not ALL_USERS:
        print("No users available for purchase")
        return pd.DataFrame()

    purchases = []
    for _ in range(num_purchases):
        user = random.choice(ALL_USERS)
        book_id, book_title, book_price, book_editorial, book_genre, book_author, book_isbn, book_page_length, book_language, book_mode = fetch_random_book()
        purchase_source = random.choice(['IOS', 'Android', 'Website'])
        purchase_date = set_random_date(year_start=2019, year_end=2024)
        payment_type = random.choice(['DEBIT', 'CREDIT', 'BOOK_CARD_CREDIT'])
        payment_status = random.choice(
            ['FAILED', 'SUCCESS', 'FRAUD', 'PENDING', 'NO_FUNDS', 'SUCCESS', 'SUCCESS', 'SUCCESS'])
        purchase_id = str(uuid.uuid4())
        money_currency = random.choice(['USD', 'MXN'])
        purchase_revenue = book_price

        if money_currency == 'USD':
            purchase_revenue = set_exchange_usd_to_mxn(book_price)

        purchase = {
            'purchase_id': purchase_id,
            'user_id': user['user_id'],
            'user_name': user['user_name'],
            'user_lastname': user['user_lastname'],
            'book_id': book_id,
            'book_title': book_title,
            'book_price': book_price,
            'user_country': user['user_country'],
            'book_editorial': book_editorial,
            'user_city': user['user_city'],
            'user_age': user['user_age'],
            'purchase_source': purchase_source,
            'book_genre': book_genre,
            'book_author': book_author,
            'book_isbn': book_isbn,
            'purchase_date': purchase_date,
            'payment_type': payment_type,
            'payment_status': payment_status,
            'purchase_revenue': purchase_revenue,
            'money_currency': money_currency,
            'book_page_length':book_page_length,
            'book_language':book_language,
            'book_mode':book_mode,
        }
        purchases.append(purchase)
    return pd.DataFrame(purchases)


def add_user_at_random_interval(num_users: int):
    while len(ALL_USERS) < num_users:
        new_user = fetch_random_user()
        if new_user not in ALL_USERS:
            ALL_USERS.append(new_user)


def register_users(num_users):
    add_user_at_random_interval(num_users=num_users)
    cols = ALL_USERS[0].keys()
    users_df = pd.DataFrame(columns=cols, data=ALL_USERS)
    return users_df

In [156]:
num_users_to_register = 5
num_purchases_to_simulate = 5

In [157]:
users_df = register_users(num_users_to_register)
users_df.head(10)

,user_id,user_name,user_lastname,user_country,user_age,user_city,user_registration_date
0,54af1f2e-6905-487d-a97d-99fda2ca3534,Flavio,Corrales,Mexico,44,Zacamulpa,2020-03-13 14:46:29
1,fbfa0226-9726-435c-8360-655957793b5a,Vincent,Pearson,Ireland,59,Carrick-on-Shannon,2020-02-20 07:30:48
2,f4504d37-1f36-43f9-993f-cf05690aeed8,Vickie,Fletcher,Australia,78,Bundaberg,2019-02-05 06:40:22
3,7d9d190d-155d-4909-923e-06e22232e4e0,Konsta,Saarinen,Finland,56,Parikkala,2023-09-26 18:44:18
4,e81ec1e2-6869-4a2d-b2af-db3ab8e253b8,Ülkü,Kulaksızoğlu,Turkey,65,Batman,2022-07-27 09:52:37


In [158]:
purchases_df = simulate_book_purchases(10)

In [159]:
purchases_df.head(10)

,purchase_id,user_id,user_name,user_lastname,book_id,book_title,book_price,user_country,book_editorial,user_city,...,book_author,book_isbn,purchase_date,payment_type,payment_status,purchase_revenue,money_currency,book_page_length,book_language,book_mode
0,bb8f2520-66b8-48ad-a1c4-15fba0b68fb7,fbfa0226-9726-435c-8360-655957793b5a,Vincent,Pearson,PjmQnVX4OmUC,Chemical Solution Deposition Of Semiconductor ...,26.825186,Ireland,CRC Press,Carrick-on-Shannon,...,Gary Hodes,0203909097,2022-01-02 06:11:24,BOOK_CARD_CREDIT,SUCCESS,26.825186,MXN,400,en,AUDIO_BOOK
1,2ff53bca-7537-4445-8fc7-2562a65b1b5f,54af1f2e-6905-487d-a97d-99fda2ca3534,Flavio,Corrales,ND6ZbQabGxcC,Daniel's Mystery Egg,41.741940,Mexico,Houghton Mifflin Harcourt,Zacamulpa,...,Alma Flor Ada,0152059660,2020-02-07 22:09:20,DEBIT,NO_FUNDS,41.741940,MXN,32,es,PDF
2,859d7d75-b58c-406b-b576-623c11889cc2,54af1f2e-6905-487d-a97d-99fda2ca3534,Flavio,Corrales,uZ0wDwAAQBAJ,Ethno-erotic Economies,8.542831,Mexico,University of Chicago Press,Zacamulpa,...,George Paul Meiu,9780226491202,2019-12-27 21:27:27,DEBIT,SUCCESS,140.860000,USD,324,en,PHYSICAL
3,743f186f-cada-4e41-9cd2-5be08bf902ff,fbfa0226-9726-435c-8360-655957793b5a,Vincent,Pearson,BWDZm8HdopYC,El robo del Santo Grial (The Mystery Team. Caz...,20.553664,Ireland,MONTENA,Carrick-on-Shannon,...,Varios autores,9788425348273,2021-11-16 12:14:22,DEBIT,NO_FUNDS,338.890000,USD,116,es,PHYSICAL
4,4082d52c-e6e1-4e24-9646-3b68031a4600,7d9d190d-155d-4909-923e-06e22232e4e0,Konsta,Saarinen,EquizXeXb3kC,John Fowles's Fiction and the Poetics of Postm...,42.172349,Finland,Associated University Presse,Parikkala,...,Mahmoud Salami,083863446X,2022-01-17 16:57:01,CREDIT,SUCCESS,695.350000,USD,312,en,PDF
5,b2a49f6f-1b00-4faa-b2f5-db7ba8574031,e81ec1e2-6869-4a2d-b2af-db3ab8e253b8,Ülkü,Kulaksızoğlu,IDO4EAAAQBAJ,30 Fitness Recipes for Healthy and Inexpensive...,12.061124,Turkey,Jhonny Sanchez,Batman,...,Jhon Boric,Unknown,2022-02-20 23:56:49,DEBIT,SUCCESS,198.870000,USD,35,es,PHYSICAL
6,4696bc29-053b-4898-9bd6-aaf09eeecefc,e81ec1e2-6869-4a2d-b2af-db3ab8e253b8,Ülkü,Kulaksızoğlu,R1IuJs5hwy0C,History in a new frontier,46.292569,Turkey,Univ de Castilla La Mancha,Batman,...,Association for History and Computing. Interna...,8484270416,2019-03-15 06:12:00,DEBIT,NO_FUNDS,46.292569,MXN,440,es,PHYSICAL
7,ab485553-339c-4708-a174-277dbf9701e7,f4504d37-1f36-43f9-993f-cf05690aeed8,Vickie,Fletcher,oh-5AAmboMUC,Russia and the Russians,19.340720,Australia,Harvard University Press,Bundaberg,...,Geoffrey A. Hosking,0674004736,2020-08-29 15:25:52,BOOK_CARD_CREDIT,SUCCESS,318.900000,USD,776,en,EBOOK
8,ba20b5c2-396c-47e4-bf7b-31d4d46033a3,54af1f2e-6905-487d-a97d-99fda2ca3534,Flavio,Corrales,RGoRBnBS8OQC,Erotic Welfare,26.026493,Mexico,Psychology Press,Zacamulpa,...,Linda Singer,0415902029,2023-09-02 07:56:53,DEBIT,SUCCESS,26.026493,MXN,220,en,EBOOK
9,f9958eb1-5bf5-496f-86d1-838beb49c750,e81ec1e2-6869-4a2d-b2af-db3ab8e253b8,Ülkü,Kulaksızoğlu,_iWAgrcajqIC,El misterio de las momias (The Mystery Team. C...,6.061873,Turkey,MONTENA,Batman,...,Varios autores,9788484419433,2021-01-03 10:48:29,BOOK_CARD_CREDIT,SUCCESS,6.061873,MXN,102,es,PHYSICAL
